# Clean

In [44]:
%%time
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd, numpy as np, datetime as dt, json
from scipy.optimize import curve_fit
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', 999)
ts_str = '2020_09_15_15_18_23'
df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
print(df.shape, ', '.join(df.columns))

(285054, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 986 ms


In [2]:
print(df.TransactionType.value_counts())
df = df.loc[df.TransactionType=='ชั่งซื้อ']
print(df.shape, 1)
df = df.loc[df.SupplierName.notnull()]
print(df.shape, 2)
df = df.loc[df.MillWeight>0]
print(df.shape, 3)
df = df.loc[df.Purchase_price_thb_ton>0]
print(df.shape, 4)
df = df.loc[df.Price_ton>0]
print(df.shape, 5)
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
print(df.shape, 6)

ชั่งซื้อ       284961
รับจ้างชั่ง        87
ชั่งขาย             5
อื่นๆ               1
Name: TransactionType, dtype: int64
(284961, 19) 1
(284959, 19) 2
(284905, 19) 3
(263200, 19) 4
(263200, 19) 5
(189479, 19) 6


In [3]:
df = df.loc[~df.Sloc.isin(['W1NS','W1CP','N1BK'])]
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','บางกระทุ่ม','ตาก', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]
print(df.shape)

(189088, 20)


In [4]:
sloc = dict(df.groupby('SiteName')['Sloc'].max())
sloc = dict([(value, key) for key, value in sloc.items()])
zone = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

In [5]:
ck = ['Zone', 'Sloc','Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton', 'SupplierName']
df = df.drop(df.loc[(df.Year == dt.datetime.now().year) & (df.Month == dt.datetime.now().month)].index)[ck]
df = df.loc[df.Year>=2016].reset_index(drop=True)
print(df.shape) # WorkCentreName, 'Time', 'Ticket_no', SiteName
df.Year = df.Year.astype('str') # Change year type to string
year = dt.datetime.now().year
years = [str(year) for year in range(year-4,year+1)]

(173074, 9)


# Supply

In [6]:
supply = df.copy()
supply = supply.groupby(['Sloc','Year','Month','Price_ton']).agg({'MillWeight':'sum', 'Purchase_price_thb_ton':'sum', 'SupplierName':'nunique', 'Day':'count'}).reset_index()
supply = supply.rename(columns={'Day':'Num_Tran'}).sort_values(by=['Year','Month','Price_ton']).reset_index(drop=True) # , ascending=False

In [7]:
supply_= supply.copy()
supply = {}
for s in sloc:
    supply[s] = {}
    for y in years:
        supply[s][y] = {}
        for m in range(1,13):
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                supply[s][y][m] = supply_.loc[(supply_.Sloc==s)&(supply_.Year==y)&(supply_.Month==m)]
                supply[s][y][m] = supply[s][y][m].merge(supply[s][y][m][['MillWeight', 'Purchase_price_thb_ton']].cumsum(),
                                                        left_index=True, right_index=True, suffixes=('','_c'))

In [8]:
supply['CH11']['2020'][1]

,Sloc,Year,Month,Price_ton,MillWeight,Purchase_price_thb_ton,SupplierName,Num_Tran,MillWeight_c,Purchase_price_thb_ton_c
1894,CH11,2020,1,"1,000.0000","7,290.3800","7,290,380.0000",416,3023,"7,290.3800","7,290,380.0000"
1899,CH11,2020,1,"1,010.0000",119.8900,"121,088.9000",4,40,"7,410.2700","7,411,468.9000"
1903,CH11,2020,1,"1,020.0000",159.3500,"162,537.0000",2,66,"7,569.6200","7,574,005.9000"
1911,CH11,2020,1,"1,050.0000",104.1600,"109,368.0000",1,19,"7,673.7800","7,683,373.9000"
1921,CH11,2020,1,"1,150.0000",169.9400,"195,431.0000",1,30,"7,843.7200","7,878,804.9000"
1922,CH11,2020,1,"1,160.0000",412.1400,"478,082.4000",5,41,"8,255.8600","8,356,887.3000"
1924,CH11,2020,1,"1,170.0000","1,400.1500","1,638,175.5000",1,345,"9,656.0100","9,995,062.8000"
1925,CH11,2020,1,"1,180.0000","1,627.4200","1,920,355.6000",7,90,"11,283.4300","11,915,418.4000"
1927,CH11,2020,1,"1,190.0000",283.1800,"336,984.2000",2,30,"11,566.6100","12,252,402.6000"
1928,CH11,2020,1,"1,200.0000",753.0000,"903,600.0000",1,34,"12,319.6100","13,156,002.6000"


In [9]:
month_valu, month_quan, month_price = {}, {}, {}
for s in sloc:
    month_valu[s], month_quan[s], month_price[s] = {}, {}, {}
    for y in years:
        month_valu[s][y], month_quan[s][y], month_price[s][y] = {}, {}, {}
        for m in range(1,13):
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                month_valu[s][y][m] = supply[s][y][m].loc[(supply[s][y][m].Year==y)&(supply[s][y][m].Month==m)].Purchase_price_thb_ton.sum()
                month_quan[s][y][m] = supply[s][y][m].loc[(supply[s][y][m].Year==y)&(supply[s][y][m].Month==m)].MillWeight.sum()
                month_price[s][y][m]= month_valu[s][y][m]/month_quan[s][y][m]
                print(f'{s}-{y}-{m}, month_valu={month_valu[s][y][m]:,.2f}, month_quan={month_quan[s][y][m]:,.2f}, month_price={month_price[s][y][m]:,.2f}')

N3CD-2016-1, month_valu=830,963.00, month_quan=826.46, month_price=1,005.45
N3CD-2016-2, month_valu=609,070.00, month_quan=609.07, month_price=1,000.00
N3CD-2016-3, month_valu=734,800.50, month_quan=756.47, month_price=971.35
N3CD-2016-4, month_valu=461,880.50, month_quan=486.19, month_price=950.00
N3CD-2016-5, month_valu=446,399.30, month_quan=461.03, month_price=968.27
N3CD-2016-6, month_valu=364,448.40, month_quan=375.72, month_price=970.00
N3CD-2016-7, month_valu=125,818.70, month_quan=129.71, month_price=970.00
N3CD-2016-8, month_valu=393,654.00, month_quan=352.46, month_price=1,116.88
N3CD-2016-9, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2016-10, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2016-11, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2016-12, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2017-1, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2017-2, month_valu=0.00, month_quan=0.00, month_price=nan
N3CD-2017-3, month_va

<ipython-input-9-a2519b3d29a2>:10: RuntimeWarning: invalid value encountered in double_scalars
  month_price[s][y][m]= month_valu[s][y][m]/month_quan[s][y][m]


N1DK-2017-2, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2017-3, month_valu=5,380.00, month_quan=5.38, month_price=1,000.00
N1DK-2017-4, month_valu=3,706.50, month_quan=3.53, month_price=1,050.00
N1DK-2017-5, month_valu=20,485.50, month_quan=19.51, month_price=1,050.00
N1DK-2017-6, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2017-7, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2017-8, month_valu=8,673.00, month_quan=8.26, month_price=1,050.00
N1DK-2017-9, month_valu=15,787.00, month_quan=15.46, month_price=1,021.15
N1DK-2017-10, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2017-11, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2017-12, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2018-1, month_valu=0.00, month_quan=0.00, month_price=nan
N1DK-2018-2, month_valu=2,023.50, month_quan=2.13, month_price=950.00
N1DK-2018-3, month_valu=6,393.50, month_quan=6.73, month_price=950.00
N1DK-2018-4, month_valu=19,824.00, month_quan=21.00,

W1DT-2020-1, month_valu=30,175.20, month_quan=30.48, month_price=990.00
W1DT-2020-2, month_valu=43,857.00, month_quan=44.30, month_price=990.00
W1DT-2020-3, month_valu=0.00, month_quan=0.00, month_price=nan
W1DT-2020-4, month_valu=0.00, month_quan=0.00, month_price=nan
W1DT-2020-5, month_valu=0.00, month_quan=0.00, month_price=nan
W1DT-2020-6, month_valu=0.00, month_quan=0.00, month_price=nan
W1DT-2020-7, month_valu=0.00, month_quan=0.00, month_price=nan
W1DT-2020-8, month_valu=59,083.20, month_quan=59.68, month_price=990.00
N1TN-2016-1, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-2, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-3, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-4, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-5, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-6, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-7, month_valu=0.00, month_quan=0.00, month_price=nan
N1TN-2016-8, month_valu=0.00

N3TK-2018-10, month_valu=0.00, month_quan=0.00, month_price=nan
N3TK-2018-11, month_valu=6,350.00, month_quan=6.35, month_price=1,000.00
N3TK-2018-12, month_valu=0.00, month_quan=0.00, month_price=nan
N3TK-2019-1, month_valu=1,940,389.50, month_quan=1,847.99, month_price=1,050.00
N3TK-2019-2, month_valu=1,512,812.50, month_quan=1,468.75, month_price=1,030.00
N3TK-2019-3, month_valu=947,339.60, month_quan=937.96, month_price=1,010.00
N3TK-2019-4, month_valu=544,551.60, month_quan=539.16, month_price=1,010.00
N3TK-2019-5, month_valu=369,134.80, month_quan=365.48, month_price=1,010.00
N3TK-2019-6, month_valu=349,288.30, month_quan=345.83, month_price=1,010.00
N3TK-2019-7, month_valu=310,736.00, month_quan=317.75, month_price=977.93
N3TK-2019-8, month_valu=287,954.50, month_quan=303.11, month_price=950.00
N3TK-2019-9, month_valu=85,585.50, month_quan=90.09, month_price=950.00
N3TK-2019-10, month_valu=60,819.00, month_quan=64.02, month_price=950.00
N3TK-2019-11, month_valu=112,271.00, month

...

N1YS-2017-4, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-5, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-6, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-7, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-8, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-9, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-10, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-11, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2017-12, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-1, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-2, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-3, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-4, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-5, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-6, month_valu=0.00, month_quan=0.00, month_price=nan
N1YS-2018-7, month_valu=6,702.00, month_quan=6.97, m

N1WT-2018-8, month_valu=136,241.10, month_quan=139.32, month_price=977.90
N1WT-2018-9, month_valu=108,682.00, month_quan=109.05, month_price=996.63
N1WT-2018-10, month_valu=17,926.50, month_quan=18.87, month_price=950.00
N1WT-2018-11, month_valu=100,411.50, month_quan=95.63, month_price=1,050.00
N1WT-2018-12, month_valu=139,230.00, month_quan=132.60, month_price=1,050.00
N1WT-2019-1, month_valu=316,564.50, month_quan=301.49, month_price=1,050.00
N1WT-2019-2, month_valu=109,158.00, month_quan=103.96, month_price=1,050.00
N1WT-2019-3, month_valu=264,694.50, month_quan=252.09, month_price=1,050.00
N1WT-2019-4, month_valu=106,195.60, month_quan=109.48, month_price=970.00
N1WT-2019-5, month_valu=243,411.80, month_quan=250.94, month_price=970.00
N1WT-2019-6, month_valu=114,630.00, month_quan=114.63, month_price=1,000.00
N1WT-2019-7, month_valu=83,975.00, month_quan=89.11, month_price=942.37
N1WT-2019-8, month_valu=24,597.00, month_quan=27.33, month_price=900.00
N1WT-2019-9, month_valu=3,654.

N2SB-2019-1, month_valu=875,009.10, month_quan=830.15, month_price=1,054.04
N2SB-2019-2, month_valu=1,878,266.80, month_quan=1,823.56, month_price=1,030.00
N2SB-2019-3, month_valu=2,376,164.00, month_quan=2,375.52, month_price=1,000.27
N2SB-2019-4, month_valu=1,433,153.20, month_quan=1,432.89, month_price=1,000.18
N2SB-2019-5, month_valu=1,007,500.00, month_quan=1,007.50, month_price=1,000.00
N2SB-2019-6, month_valu=532,777.80, month_quan=517.26, month_price=1,030.00
N2SB-2019-7, month_valu=1,137,126.50, month_quan=1,171.58, month_price=970.59
N2SB-2019-8, month_valu=479,567.50, month_quan=530.71, month_price=903.63
N2SB-2019-9, month_valu=319,019.50, month_quan=335.81, month_price=950.00
N2SB-2019-10, month_valu=544,580.00, month_quan=544.58, month_price=1,000.00
N2SB-2019-11, month_valu=627,015.00, month_quan=641.20, month_price=977.88
N2SB-2019-12, month_valu=607,244.00, month_quan=606.96, month_price=1,000.47
N2SB-2020-1, month_valu=926,437.10, month_quan=991.95, month_price=933.96

N1NK-2016-9, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2016-10, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2016-11, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2016-12, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2017-1, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2017-2, month_valu=0.00, month_quan=0.00, month_price=nan
N1NK-2017-3, month_valu=64,922.00, month_quan=59.02, month_price=1,100.00
N1NK-2017-4, month_valu=53,006.40, month_quan=49.08, month_price=1,080.00
N1NK-2017-5, month_valu=25,131.60, month_quan=23.27, month_price=1,080.00
N1NK-2017-6, month_valu=36,266.40, month_quan=33.58, month_price=1,080.00
N1NK-2017-7, month_valu=13,402.80, month_quan=12.41, month_price=1,080.00
N1NK-2017-8, month_valu=25,671.60, month_quan=23.77, month_price=1,080.00
N1NK-2017-9, month_valu=40,208.40, month_quan=37.23, month_price=1,080.00
N1NK-2017-10, month_valu=15,649.20, month_quan=14.49, month_price=1,080.00
N1NK-2017-11, month_valu=6

CH12-2017-9, month_valu=9,014,611.54, month_quan=6,724.37, month_price=1,340.59
CH12-2017-10, month_valu=4,217,290.90, month_quan=3,258.08, month_price=1,294.41
CH12-2017-11, month_valu=5,322,464.40, month_quan=4,181.58, month_price=1,272.84
CH12-2017-12, month_valu=10,616,321.70, month_quan=8,565.92, month_price=1,239.37
CH12-2018-1, month_valu=7,692,557.70, month_quan=6,617.34, month_price=1,162.48
CH12-2018-2, month_valu=4,404,286.60, month_quan=3,846.59, month_price=1,144.98
CH12-2018-3, month_valu=1,070,456.10, month_quan=943.16, month_price=1,134.97
CH12-2018-4, month_valu=1,723,455.00, month_quan=1,637.41, month_price=1,052.55
CH12-2018-5, month_valu=2,310,062.00, month_quan=2,187.02, month_price=1,056.26
CH12-2018-6, month_valu=3,814,411.40, month_quan=3,523.79, month_price=1,082.47
CH12-2018-7, month_valu=8,470,519.30, month_quan=7,548.31, month_price=1,122.17
CH12-2018-8, month_valu=8,348,404.10, month_quan=7,527.80, month_price=1,109.01
CH12-2018-9, month_valu=4,323,286.10, 

In [10]:
xx, yy, width, value = {}, {}, {}, {}
for s in sloc:
    xx[s], yy[s], width[s], value[s] = {}, {}, {}, {}
    for y in years:
        xx[s][y], yy[s][y], width[s][y], value[s][y] = {}, {}, {}, {}
        for m in range(1,13):
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                xx[s][y][m] = []
                yy[s][y][m] = supply[s][y][m].loc[(supply[s][y][m].Year==y)&(supply[s][y][m].Month==m)].Price_ton.tolist()
                width[s][y][m] = supply[s][y][m].loc[(supply[s][y][m].Year==y)&(supply[s][y][m].Month==m)].MillWeight.tolist()
                value[s][y][m] = supply[s][y][m].loc[(supply[s][y][m].Year==y)&(supply[s][y][m].Month==m)].Purchase_price_thb_ton.tolist()
                for i in range(len(width[s][y][m])):
                    if i:
                        xx[s][y][m] = xx[s][y][m] + [xx[s][y][m][i-1] + width[s][y][m][i-1]/2 + width[s][y][m][i]/2]
                    else:
                        xx[s][y][m] = xx[s][y][m] + [width[s][y][m][i]/2]
                print(f'{s}-{y}-{m}', len(width[s][y][m]), len(xx[s][y][m]))

N3CD-2016-1 2 2
N3CD-2016-2 1 1
N3CD-2016-3 2 2
N3CD-2016-4 1 1
N3CD-2016-5 2 2
N3CD-2016-6 1 1
N3CD-2016-7 1 1
N3CD-2016-8 2 2
N3CD-2016-9 0 0
N3CD-2016-10 0 0
N3CD-2016-11 0 0
N3CD-2016-12 0 0
N3CD-2017-1 0 0
N3CD-2017-2 0 0
N3CD-2017-3 0 0
N3CD-2017-4 0 0
N3CD-2017-5 0 0
N3CD-2017-6 0 0
N3CD-2017-7 0 0
N3CD-2017-8 0 0
N3CD-2017-9 0 0
N3CD-2017-10 0 0
N3CD-2017-11 0 0
N3CD-2017-12 0 0
N3CD-2018-1 0 0
N3CD-2018-2 0 0
N3CD-2018-3 0 0
N3CD-2018-4 0 0
N3CD-2018-5 0 0
N3CD-2018-6 0 0
N3CD-2018-7 1 1
N3CD-2018-8 2 2
N3CD-2018-9 2 2
N3CD-2018-10 1 1
N3CD-2018-11 1 1
N3CD-2018-12 1 1
N3CD-2019-1 2 2
N3CD-2019-2 3 3
N3CD-2019-3 4 4
N3CD-2019-4 2 2
N3CD-2019-5 2 2
N3CD-2019-6 2 2
N3CD-2019-7 2 2
N3CD-2019-8 1 1
N3CD-2019-9 1 1
N3CD-2019-10 1 1
N3CD-2019-11 1 1
N3CD-2019-12 1 1
N3CD-2020-1 4 4
N3CD-2020-2 4 4
N3CD-2020-3 9 9
N3CD-2020-4 5 5
N3CD-2020-5 5 5
N3CD-2020-6 3 3
N3CD-2020-7 4 4
N3CD-2020-8 4 4
N1DK-2016-1 0 0
N1DK-2016-2 0 0
N1DK-2016-3 0 0
N1DK-2016-4 0 0
N1DK-2016-5 0 0
N1DK-2016-6 

N1PR-2020-2 2 2
N1PR-2020-3 2 2
N1PR-2020-4 2 2
N1PR-2020-5 2 2
N1PR-2020-6 0 0
N1PR-2020-7 1 1
N1PR-2020-8 1 1
N1PB-2016-1 0 0
N1PB-2016-2 0 0
N1PB-2016-3 0 0
N1PB-2016-4 0 0
N1PB-2016-5 0 0
N1PB-2016-6 0 0
N1PB-2016-7 0 0
N1PB-2016-8 0 0
N1PB-2016-9 0 0
N1PB-2016-10 0 0
N1PB-2016-11 0 0
N1PB-2016-12 0 0
N1PB-2017-1 0 0
N1PB-2017-2 0 0
N1PB-2017-3 1 1
N1PB-2017-4 1 1
N1PB-2017-5 1 1
N1PB-2017-6 1 1
N1PB-2017-7 0 0
N1PB-2017-8 1 1
N1PB-2017-9 1 1
N1PB-2017-10 0 0
N1PB-2017-11 0 0
N1PB-2017-12 0 0
N1PB-2018-1 1 1
N1PB-2018-2 1 1
N1PB-2018-3 1 1
N1PB-2018-4 2 2
N1PB-2018-5 1 1
N1PB-2018-6 0 0
N1PB-2018-7 1 1
N1PB-2018-8 1 1
N1PB-2018-9 2 2
N1PB-2018-10 2 2
N1PB-2018-11 1 1
N1PB-2018-12 1 1
N1PB-2019-1 1 1
N1PB-2019-2 1 1
N1PB-2019-3 1 1
N1PB-2019-4 1 1
N1PB-2019-5 1 1
N1PB-2019-6 1 1
N1PB-2019-7 2 2
N1PB-2019-8 1 1
N1PB-2019-9 1 1
N1PB-2019-10 1 1
N1PB-2019-11 2 2
N1PB-2019-12 2 2
N1PB-2020-1 2 2
N1PB-2020-2 3 3
N1PB-2020-3 3 3
N1PB-2020-4 2 2
N1PB-2020-5 3 3
N1PB-2020-6 3 3
N1PB-2020-7 

N3NB-2019-4 1 1
N3NB-2019-5 1 1
N3NB-2019-6 1 1
N3NB-2019-7 2 2
N3NB-2019-8 1 1
N3NB-2019-9 1 1
N3NB-2019-10 1 1
N3NB-2019-11 1 1
N3NB-2019-12 1 1
N3NB-2020-1 3 3
N3NB-2020-2 4 4
N3NB-2020-3 5 5
N3NB-2020-4 3 3
N3NB-2020-5 3 3
N3NB-2020-6 2 2
N3NB-2020-7 3 3
N3NB-2020-8 3 3
N1NK-2016-1 0 0
N1NK-2016-2 0 0
N1NK-2016-3 0 0
N1NK-2016-4 0 0
N1NK-2016-5 0 0
N1NK-2016-6 0 0
N1NK-2016-7 0 0
N1NK-2016-8 0 0
N1NK-2016-9 0 0
N1NK-2016-10 0 0
N1NK-2016-11 0 0
N1NK-2016-12 0 0
N1NK-2017-1 0 0
N1NK-2017-2 0 0
N1NK-2017-3 1 1
N1NK-2017-4 1 1
N1NK-2017-5 1 1
N1NK-2017-6 1 1
N1NK-2017-7 1 1
N1NK-2017-8 1 1
N1NK-2017-9 1 1
N1NK-2017-10 1 1
N1NK-2017-11 1 1
N1NK-2017-12 1 1
N1NK-2018-1 1 1
N1NK-2018-2 1 1
N1NK-2018-3 1 1
N1NK-2018-4 0 0
N1NK-2018-5 0 0
N1NK-2018-6 1 1
N1NK-2018-7 1 1
N1NK-2018-8 2 2
N1NK-2018-9 2 2
N1NK-2018-10 2 2
N1NK-2018-11 1 1
N1NK-2018-12 2 2
N1NK-2019-1 1 1
N1NK-2019-2 1 1
N1NK-2019-3 2 2
N1NK-2019-4 1 1
N1NK-2019-5 1 1
N1NK-2019-6 1 1
N1NK-2019-7 2 2
N1NK-2019-8 1 1
N1NK-2019-9 

In [40]:
s='CH11' #'' CH13

In [41]:
m=1

In [42]:
for y in years:
    sum_value = sum(value[s][y][m])
    fig = go.Figure(data=[go.Bar(
        x=xx[s][y][m],
        y=yy[s][y][m],
        width=width[s][y][m],
        text=value[s][y][m],
    )])
    fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
    #fig.update_xaxes(dtick=1000)
    fig.update_layout(title=f'{s}-{y}-{m} Total Value: THB {sum_value:,.2f}')
    fig.layout.title.font=dict(size=14)
    fig.show()

In [14]:
supply_years = {}
for s in sloc:
    supply_years[s] = {}
    for m in range(1,13):
        supply_years[s][m] = pd.DataFrame(columns=['Year','MillWeight_c', 'Purchase_price_thb_ton_c', 'Price_ton'])
        for y in years:
            #print(f'{y}-{m}')
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                supply_years[s][m] = supply_years[s][m].append(supply[s][y][m][['Year','MillWeight_c', 'Purchase_price_thb_ton_c', 'Price_ton']])

In [15]:
s

'CH13'

In [16]:
m=1

In [17]:
supply_years[s][m].loc[supply_years[s][m].Year=='2020']

,Year,MillWeight_c,Purchase_price_thb_ton_c,Price_ton
1882,2020,49.2600,"45,811.8000",930.0000
1910,2020,229.9600,"233,739.8000","1,040.0000"
1932,2020,259.8400,"278,559.8000","1,500.0000"


In [18]:
fig = px.scatter(supply_years[s][m].loc[supply_years[s][m].Year=='2020'], x='MillWeight_c', y='Price_ton', color='Year', title=f'{s}-{m}')
fig.show()

In [43]:
fig = px.scatter(supply_years[s][m].loc[supply_years[s][m].Year=='2020'], x='MillWeight_c', y='Price_ton', color='Year', title=f'{s}-{m}')
fig.show()

In [19]:
s

'CH13'

In [20]:
fig = px.scatter(supply_years[s][m], x='MillWeight_c', y='Price_ton', color='Year', title=f'{s}-{m}')
fig.show()

In [21]:
fig, results = {}, {}
for s in list(sloc)[:1]:
    fig[s], results[s] = {}, {}
    for m in range(1,13):
        fig[s][m] = px.scatter(supply_years[s][m], x='MillWeight_c', y='Price_ton', trendline='ols', title=f'{s}-{m}') # , color='Year'  
        results[s][m] = px.get_trendline_results(fig[s][m])
        fig[s][m].show()

d:\wp\venv\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars



In [22]:
fig, results = {}, {}
for s in list(sloc)[:1]:
    fig[s], results[s] = {}, {}
    for m in range(1,13):
        fig[s][m] = px.scatter(supply_years[s][m].loc[supply_years[s][m].Year=='2020'],
                               x='MillWeight_c', y='Purchase_price_thb_ton_c', trendline='ols', title=f'{s}-{m}') # , color='Year'  
        results[s][m] = px.get_trendline_results(fig[s][m])
        fig[s][m].show()

In [23]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
results[s][1].px_fit_results.iloc[0].summary()

d:\wp\venv\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning:

omni_normtest is not valid with less than 8 observations; 4 samples were given.



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.686e+05
Date:                Thu, 17 Sep 2020   Prob (F-statistic):           2.71e-06
Time:                        15:38:52   Log-Likelihood:                -30.299
No. Observations:                   4   AIC:                             64.60
Df Residuals:                       2   BIC:                             63.37
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.292e+04   1290.204    -10.013      0.010   -1.85e+04   -7367.679
x1          1023.1697      1.685    607.127      0.000    1015.919    1030.421
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.657
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.613
Skew:                          -0.861   Prob(JB):                        0.736
Kurtosis:                       2.154   Cond. No.                     2.96e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.96e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
results[s][1].px_fit_results.iloc[0].params

array([-12918.977504, 1023.169697])

In [25]:
from scipy.optimize import linprog

In [26]:
xxx=np.arange(1,100)

In [38]:
yyy=np.power(xxx, 1.5)

In [39]:
px.scatter(x=xxx, y=yyy)